In [ ]:
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import GRPOConfig, GRPOTrainer
import numpy as np 

In [ ]:
# tensorboard --port 6007 --logdir /root/tf-logs/
# ps -ef | grep tensorboard | awk '{print $2}' | xargs kill -9 
# tensorboard --logdir=/mnt/data2/icd10/outputs/Qwen-4B-instruct/runs/Oct08_16-58-14_CathayBrain--port=6007
# tensorboard --logdir "/mnt/data2/icd10/outputs/Qwen-4B-instruct/runs/Oct08_16-30-04_CathayBrain"

# autodl-tmp/outputs/Qwen-0.5B-GRPO/runs/Apr16_16-34-38_autodl-container-ef014e83a6-5e7d45d3

In [ ]:
def extract_icd_answer(text: str) -> str:
    """
    從模型的完整回覆或標準答案中提取 ICD-10 編碼。
    優先從 <answer> 標籤中提取,若無則從整個文本中提取。
    """
    # 使用正則表達式搜尋 <answer> 標籤，re.DOTALL 讓 . 可匹配換行符，re.IGNORECASE 忽略大小寫
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL | re.IGNORECASE)
    # 如果找到 <answer> 標籤，則只在標籤內容中搜尋 ICD 編碼；否則在整個文本中搜尋
    search_text = answer_match.group(1) if answer_match else text
    
    # 匹配格式：字母 + 1-3位數字 + 可選的小數點和1-2位數字
    codes_list = re.findall(r'\b([A-Z]\d{1,6}(?:\.\d{1,2})?)\b', search_text, re.IGNORECASE)
    
    # 移除小數點、去重、排序、轉大寫，並過濾掉長度<=2的編碼
    cleaned_codes = sorted(set(
        code.replace('.', '').upper() 
        for code in codes_list 
        if len(code.replace('.', '')) > 2
    ))
    return " ".join(cleaned_codes)

In [ ]:
import csv
import re

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
請嚴格遵守以下作答格式：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結出最合理的編碼集合。  
</reasoning>  
<answer>  
僅輸出最終的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""
EXAMPLE_QUESTION = """
Sex: F
Service: MEDICINE
Allergies: Aspirin
Attending: ___
Chief Complaint: weakness, diarrhea
Major Surgical or Invasive Procedure: None

History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for chronic anemia, osteoporosis, hypertension, ataxia, and recent L5 fracture in the setting of recurrent falls who presents from home with fatigue and generalized weakness and diarrhea.  

The patient's recent history is notable for the follow:  
- On ___, she presented with 4 days of LBP s/p fall from standing at which time imaging revealed acute L5 fracture. She was evaluated by Spine team who recommended early mobilization, pain control, but no brace required. She was evaluated by ___, and discharged to ___.  
- She was discharged home with ___ on ___.  
- On ___, she again presented to ___ s/p fall from standing while trying to reach for a glass of water. She did have a occipital scalp hematoma, but imaging including ___, C-spine CT, and L hip X-ray were negative for acute process so patient was discharged home.  

She now represents with generalized fatigue and diarrhea. In the setting of opiates for her L5 fracture, the patient has had constipation (5 days with no BM) for which she took a "natural laxative" the evening prior to presentation. The patient had 2 bowel movements in the morning of presentation and one episode of incontinence with diarrhea while sleeping. In this setting, she felt very weak and called EMS and was brought to ___ ED.

What ICD-10 codes should be assigned? Please just tell me which codes in the end
"""
EXAMPLE_REASONING = """The patient has chronic anemia, documented and clinically relevant, which corresponds to D500.  
There is vitamin deficiency suggested, which fits E538.  
She also has visual impairment history, which maps to H548.  
Hypertension is a chronic condition, coded as I10.  
Diarrhea due to laxative use is coded as K521 (toxic gastroenteritis and colitis).  
Her recent fracture with underlying osteoporosis corresponds to M810.  
Ataxia is a chronic neurologic condition, coded as R270.  
There is adverse effect of drugs (laxatives/opioids), so T474X5A applies.  
Environmental factor of injury is Y92099 (unspecified place of occurrence).  
She also has aspirin allergy, captured by Z9181.  

Therefore, the most appropriate ICD-10 codes are:
"""

EXAMPLE_ANSWER = "D500 E538 H548 I10 K521 M810 R270 T474X5A Y92099 Z9181"


chat_style_data = []

with open("results/0.4_acc_result.csv", "r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    
    for row in reader:
        # row 現在是一個字典，鍵值為 CSV 的標頭 (Header)
        chat_style_data.append({
            "prompt": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": EXAMPLE_QUESTION},
                {"role": "assistant", "content": f"<reasoning>{EXAMPLE_REASONING}</reasoning><answer>{EXAMPLE_ANSWER}</answer>"},
                {"role": "user", "content": row["question"]}  # 從 CSV 的 'question' 欄位讀取
            ],
            # 假設 CSV 內也有 'answer' 欄位供 extract_icd_answer 使用
            "answer":  row["ground_truths_ans"]
        })

In [ ]:
# 用法
dataset = chat_style_data
chat_style_data

In [ ]:
#reward中會用到的function
def extract_xml_answer(text: str) -> str: #提取出答案
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    answer = answer.strip()
    answer = answer.replace(".", "") # 移除 ICD-10 編碼中的小數點，例如 Z45.81 → Z4581
    return answer

def is_text(text: str) -> bool:
    """
    判斷輸入字串是否主要為文字敘述（而非 ICD code 等短編碼）

    text: 輸入的字串

    回傳:
        True  -> 主要是文字敘述
        False -> 主要不是文字（例如短編碼）
    """
    # 計算字串中英文字母或空白的總數
    letters_and_spaces = sum(c.isalpha() or c.isspace() for c in text)

    # 計算文字/空白比例，避免除以 0
    ratio = letters_and_spaces / max(1, len(text))  

    # 如果文字/空白比例超過 0.5，視為文字敘述
    if ratio > 0.5:  
        return True  
    else:
        return False

def compute_f1(pred_set: set[str], true_set: set[str]) -> float:
    """
    pred_set: 模型預測的 ICD-10 code 集合
    true_set: 真實答案的 ICD-10 code 集合
    """
    if not pred_set and not true_set:
        print("⚠️ Both prediction and ground truth are empty, F1=1.0")
        return 1.0
    tp = len(pred_set & true_set)   # true positive
    fp = len(pred_set - true_set)   # false positive
    fn = len(true_set - pred_set)   # false negative

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    if precision + recall == 0:
        f1 = 0.0
    else:
        f1 = 2 * precision * recall / (precision + recall)

    return f1

def strict_format_check(prediction: str, ground_truth: str) -> bool:
    """
    嚴格格式檢查，用於 reward 計算前的 F1 安全檢查
    回傳 True -> 可以安全計算 F1, False -> 無法計算 F1
    """
    try:
        # 將字串拆成集合，支援逗號或空格分隔
        def parse_codes(text):
            # 如果包含逗號，用逗號分隔；否則用空格分隔
            if ',' in text:
                return set(code.strip() for code in text.split(',') if code.strip())
            else:
                return set(code.strip() for code in text.split() if code.strip())
        
        pred_set = parse_codes(prediction)
        true_set = parse_codes(ground_truth)
        
        # 檢查格式是否符合 ICD-10（字母開頭 + 2~4 位數字）
        # 修復：調整正則表達式以符合實際的 ICD-10 格式
        icd_pattern = re.compile(r'^[A-Z][0-9A-Z]{2,6}([.][0-9A-Z]+)?$')
        
        # 檢查所有 code 是否符合格式
        for code in pred_set:
            # 保持原始格式進行檢查，不要移除點號
            if not icd_pattern.match(code):
                print(f'strict_format_check fail: {code[:20]}')
                return False
        
        # 嘗試計算 F1（檢查是否能計算，不真正使用結果）
        # 修復：將 * = compute*f1 改為正確的函數調用
        f1_score = compute_f1(pred_set, true_set)
        return True
        
    except Exception as e:
        print("Error in strict_format_check:", e)
        print('strict_format_check error')
        return False

In [ ]:
def accuracy_func(prompts, completions, answer, **kwargs) -> list[float]:
    """
    計算 F1-score 作為 reward
    規則：
    1. 如果回答是文字敘述或格式不符 ICD-10，直接給 0
    2. 只要前三碼相同就算答對
    """

    # 模型輸出文字
    responses = [completion[0]['content'] for completion in completions]

    # 取得問題（debug 用）
    q = prompts[0][-1]['content']

    # 提取 XML 中的回覆
    extracted_responses = [extract_xml_answer(r) for r in responses]

    # Debug 輸出
    print('-'*20)
    print(f"Question:\n{q}")
    print(f"\nAnswer:\n{answer[0]}")
    print(f"\nResponse:\n{responses[0]}")
    print(f"\nExtracted:\n{extracted_responses[0]}")

    f1_scores = []

    for r, a in zip(extracted_responses, answer):

        # 移除答案前綴
        if a.startswith("ICD10 編碼："):
            a = a.replace("ICD10 編碼：", "")

        # 文字敘述檢查 + 嚴格格式檢查
        if is_text(r) or not strict_format_check(r, a):
            f1_scores.append(0.0)
            continue

        # 處理答案與模型輸出
        a_codes = [code.strip()[:3] for code in a.split() if code.strip()]
        r_codes = [code.strip()[:3] for code in r.split(',') if code.strip()]

        a_set = set(a_codes)
        r_set = set(r_codes)

        # 計算 F1-score 並乘 20
        f1_scores.append(compute_f1(r_set, a_set) * 30)

    # Debug 輸出
    print('--------------------')
    print("F1 scores:", f1_scores)
    print('--------------------')

    return f1_scores

def soft_format_reward_func(completions, **kwargs) -> list[float]: #只要不是一堆文字敘述就給分
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]

    rewards = []
    for r in extracted_responses:
        if is_text(r):  
            rewards.append(0.0)   # 主要是文字 → 0 分
        else:
            rewards.append(0.5)   # 主要是短編碼 → 0.5 分
    print('soft text rewards',rewards)
    return rewards

def strict_format_reward_func(completions, answer, **kwargs) -> list[float]: #要可以分解成icd10編碼才給分
    """Reward function that checks if the completion has a specific format."""
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]

    rewards = []
    for r, a in zip(extracted_responses, answer):
        # 移除答案前綴
        if a.startswith("ICD10 編碼："):
            a = a.replace("ICD10 編碼：", "")
        # 文字敘述檢查 + 嚴格格式檢查
        if strict_format_check(r,a):  
            rewards.append(1)   # 是icd10格式 → 1 分
        else:
            rewards.append(0.0)   # 不符合格式 → 0 分
    print('strict text rewards',rewards)
    return rewards


def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    rewards = [count_xml(c) for c in contents]
    print("xml rewards:", rewards)
    return rewards


# def length_cosine_reward_func(completions, answer, prompts, **kwargs) -> list[float]:
#     responses = [completion[0]['content'] for completion in completions]
#     extracted_answers = [extract_xml_answer(r) for r in responses]

#     rewards = []
#     for i, r in enumerate(responses):  # 對於每一個模型輸出的回答 response（r）
#         reasoning_text = r.split("<answer>")[0]  # 只取出 `<answer>` 之前的部分（即 reasoning 推理內容）
        
#         # 使用 tokenizer 將 reasoning_text 分詞成 tokens，並取其長度作為 reasoning 的 token 長度
#         reasoning_len = len(tokenizer(reasoning_text, return_tensors="pt")["input_ids"][0])
        
#         is_correct = extracted_answers[i] == answer[i]  # 判斷模型的最終回答是否與正確答案相符（是否正確）

#         # 設定最大長度為 16000，將推理長度 reasoning_len 映射到 [0, π] 的範圍（正規化）
#         max_length = 786 
#         x = np.clip(reasoning_len / max_length, 0, 1) * np.pi/2  # 確保比例不超過 1，避免過長失控

#         if is_correct:
#             # 如果答對了，就使用「遞減型餘弦」，鼓勵短的答案（reward 隨長度增加而減少）
#             reward = 5 * np.cos(x)  # 最多給 +5，越長越接近 0
#         else:
#             # 如果答錯了，就使用「反向餘弦」，鼓勵多一點思考（reward 隨長度增加而上升）
#             # reward = -5.0 * np.cos(x)  # 最多扣 -10，越長越接近 0（或甚至轉為正）
#             reward = 0
#         rewards.append(float(reward))  # 把這一筆的 reward 加入總 reward 清單
#     return rewards  # 回傳所有 sample 的 reward 分數

In [ ]:
from transformers import TrainerCallback, TrainerState, TrainerControl

class CustomSaveCallback(TrainerCallback):
    def on_step_end(self, args, state: TrainerState, control: TrainerControl, **kwargs):
        if state.global_step >= 1500 and state.global_step % 100 == 0:
            control.should_save = True
        else:
            control.should_save = False
        return control


In [ ]:
# 選擇模型名稱
model_name = "models/Qwen3-14B-sft_v2"

# 設定輸出資料夾與 run_name，方便 tensorboard / log 區分不同模型
if "Llama" in model_name:
    output_dir = "outputs/Llama-1B-GRPO"
    run_name = "Llama-1B-GRPO-gsm8k"
else:
    output_dir = "outputs/Qwen-14B-sft_v2"
    run_name = "Qwen-14B-sft_v2"
    
# GRPO 訓練參數設定
training_args = GRPOConfig(
    output_dir=output_dir,         # 模型 checkpoint 與 log 的輸出位置
    run_name=run_name,             # run 名稱，用於 tensorboard 區分
    learning_rate=2e-6,            # 基本學習率
    adam_beta1=0.9,                # Adam 優化器參數 beta1
    adam_beta2=0.99,               # Adam 優化器參數 beta2
    weight_decay=0.1,              # 權重衰減，避免 overfitting
    warmup_ratio=0.1,              # 前 10% step 做 learning rate warmup
    lr_scheduler_type='cosine',    # 餘弦退火學習率
    logging_steps=1,               # 每 step log 一次
    bf16=True,                     # 使用 bfloat16 訓練（省顯存，保持精度）
    per_device_train_batch_size=1, # 每個 GPU batch size
    gradient_accumulation_steps=3, # 梯度累積，等效 batch size = 1*3=3
    num_generations=3,             # 每個 prompt 生成多少個答案，供 reward function 打分
    max_prompt_length=500,        # prompt 最長 token 長度
    max_completion_length=950,     # 模型生成部分最長 token 長度
    num_train_epochs=1,            # 訓練 epoch 數
    save_steps=100,                # 每 100 steps 存 checkpoint
    max_grad_norm=0.1,             # 梯度裁剪上限，避免梯度爆炸
    report_to="tensorboard",       # log 到 tensorboard
    log_on_each_node=False,        # 多機訓練時是否每個節點都 log
)

# LoRA 低秩適應設定，用於節省記憶體與加速訓練
peft_config = LoraConfig(
    r=16,                          # LoRA rank
    lora_alpha=64,                 # LoRA scaling 係數
    target_modules=[               # 指定要插入 LoRA 的線性層
        "q_proj", "k_proj", "v_proj", "ao_proj", 
        "up_proj", "down_proj", "gate_proj"
    ],
    task_type="CAUSAL_LM",         # 任務類型：因果語言模型
    lora_dropout=0.05,             # LoRA dropout，避免 overfitting
)

from transformers import BitsAndBytesConfig
from peft import prepare_model_for_kbit_training

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",       # 使用 NF4 分佈，對權重保留更好
    bnb_4bit_compute_dtype=torch.bfloat16, # 計算時轉回 bf16，保持精確度
    bnb_4bit_use_double_quant=True,  # 二次量化，再省約 0.4 bit/param
)

# 載入模型，啟用 Flash Attention 2
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,    # 注入量化設定
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    trust_remote_code=True,
)

# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # pad token 設為 eos token

# 初始化 GRPOTrainer
# - reward_funcs: 自訂獎勵函數清單
# - train_dataset: 訓練資料
# - peft_config: LoRA 配置（若不用 LoRA 可移除）
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        accuracy_func,             # 最終準確率獎勵
        soft_format_reward_func,   # 判斷文字敘述程度 如果不是文字敘述就加分
        strict_format_reward_func # 嚴格格式符合獎勵
    ],
    args=training_args,
    train_dataset=dataset,
    # callbacks=[CustomSaveCallback()], # 若要自定義 checkpoint 存法可打開
    peft_config=peft_config
)

# 開始訓練
trainer.train()
